# Bonus Analysis

In [1]:
# import dependancies
import os 
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from config import dbname, username, password, server
import matplotlib.pyplot as plt

In [2]:
# connect to database
engine = create_engine(f'postgresql://{username}:{password}@{server}/{dbname}')
conn= engine.connect()

In [3]:
# test connection
data = pd.read_sql("SELECT * FROM salaries", conn)
data.head()

,emp_no,salary
0,10001,60117
1,10002,65828
2,10003,40006
3,10004,40054
4,10005,78228


In [4]:
# review tables in database
# https://stackoverflow.com/questions/21310549/list-database-tables-with-sqlalchemy/21346185
m = MetaData()
m.reflect(engine)
for table in m.tables.values():
    print(table.name)

departments
dept_emp
dept_manager
employees
salaries
titles


In [5]:
# Create a histogram to visualize the most common salary ranges for employees.


In [6]:
# Create a bar chart of average salary by title.
# first get unique titles from titles table
pd.read_sql("SELECT distinct title from titles", conn)

,title
0,Engineer
1,Technique Leader
2,Manager
3,Assistant Engineer
4,Staff
5,Senior Engineer
6,Senior Staff


In [9]:
# next create query to join salaries with title tables
# look at columns for title table, salaries above
salary = pd.read_sql_table('titles', conn)
salary.head(2)

,title_id,title
0,s0001,Staff
1,s0002,Senior Staff


In [11]:
# look at columns for employees, need to join off title_id
emp = pd.read_sql_table('employees', conn)
emp.head(2)

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302,s0001,1953-07-25,Hideyuki,Zallocco,M,1990-04-28
1,475053,e0002,1954-11-18,Byong,Delgrande,F,1991-09-07
